<a href="https://colab.research.google.com/github/Annamjohn/Twitter-Sentiment-Analysis/blob/main/Sentiment%20Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Install kaggle library
! pip install kaggle

Upload your kaggle.json file

In [3]:
# configure the path of kaggle.json file

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [4]:
# fetch and import Twitter Sentiment dataset using API
!kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
 96% 78.0M/80.9M [00:03<00:00, 32.7MB/s]
100% 80.9M/80.9M [00:03<00:00, 24.9MB/s]


In [5]:
#Extract the compressed dataset

from zipfile import ZipFile
dataset= '/content/sentiment140.zip'

with ZipFile(dataset, 'r') as zip:
  zip.extractall()
  print("The dataset has been extracted")

The dataset has been extracted


In [6]:
#import the dependencies
import numpy as np
import pandas as pd
import re #pattern matching, search through data
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [7]:
import nltk
nltk.download('stopwords')
print(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Data Processing

In [8]:
# loading data from csv file to pd DF
twitter_data=pd.read_csv('/content/training.1600000.processed.noemoticon.csv', encoding= 'ISO-8859-1') #characters in your CSV file are read correctly, avoiding errors or misinterpretation of special characters. Encoding: To convert characters into bytes and vice versa.

In [9]:
#no. of rows and columns
twitter_data.shape

(1599999, 6)

In [10]:
twitter_data.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [11]:
column_names=['target','id','date','flag','user','text']
twitter_data=pd.read_csv('/content/training.1600000.processed.noemoticon.csv', names= column_names, encoding= 'ISO-8859-1')

In [12]:
twitter_data.shape

(1600000, 6)

In [13]:
twitter_data.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [14]:
twitter_data.isnull().sum() # sum of no. of null vals in each column

target    0
id        0
date      0
flag      0
user      0
text      0
dtype: int64

In [15]:
twitter_data['target'].value_counts()

target
0    800000
4    800000
Name: count, dtype: int64

In [16]:
#converting 4 to 1
twitter_data.replace({'target': {4:1}}, inplace=True) # inplace=True => changes will be applied directly to the DataFrame twitter_data without creating a new DataFrame.
twitter_data['target'].value_counts()

target
0    800000
1    800000
Name: count, dtype: int64

0=> Negative Tweet

1=> Positive Tweet

**Stemming**

In [17]:
port_stem=PorterStemmer()

In [18]:
def stemming(content):
  combined_pattern = r'http[s]?://\S+|\S+@\S+|@\w+|[^a-zA-Z\s]'

    # Remove unwanted patterns
  stemmed_content = re.sub(combined_pattern, ' ', content) # ^ =>not i.e. replace all non-alphabetic characters in content with a space
  stemmed_content= stemmed_content.lower()
  stemmed_content= stemmed_content.split() # Split the string into a list of words based on whitespace
  stemmed_content= [port_stem.stem(word) for word in stemmed_content if word not in stopwords.words('english')]
  stemmed_content= ' '.join(stemmed_content) # Joins the list of stemmed words back into a single string for each tweet, with words separated by spaces.

  return stemmed_content

In [19]:
# def stemming(content):

#     # Remove unwanted patterns
#   stemmed_content = re.sub('[^a-zA-Z]', ' ', content) # ^ =>not i.e. replace all non-alphabetic characters in content with a space
#   stemmed_content= stemmed_content.lower()
#   stemmed_content= stemmed_content.split() # Split the string into a list of words based on whitespace
#   stemmed_content= [port_stem.stem(word) for word in stemmed_content if word not in stopwords.words('english')]
#   stemmed_content= ' '.join(stemmed_content) # Joins the list of stemmed words back into a single string for each tweet, with words separated by spaces.

#   return stemmed_content

In [20]:
twitter_data['stemmed_content'] = twitter_data['text'].apply(stemming)

In [28]:
twitter_data.head()

,target,id,date,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",awww bummer shoulda got david carr third day
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,dive mani time ball manag save rest go bound
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",behav mad see


In [22]:
print(twitter_data['stemmed_content'])

0               awww bummer shoulda got david carr third day
1          upset updat facebook text might cri result sch...
2               dive mani time ball manag save rest go bound
3                            whole bodi feel itchi like fire
4                                              behav mad see
                                 ...                        
1599995                           woke school best feel ever
1599996              thewdb com cool hear old walt interview
1599997                         readi mojo makeov ask detail
1599998    happi th birthday boo alll time tupac amaru sh...
1599999                                 happi charitytuesday
Name: stemmed_content, Length: 1600000, dtype: object


In [23]:
print(twitter_data['target'])

0          0
1          0
2          0
3          0
4          0
          ..
1599995    1
1599996    1
1599997    1
1599998    1
1599999    1
Name: target, Length: 1600000, dtype: int64


In [32]:
# seperate data and label

X= twitter_data['stemmed_content'].values
Y=twitter_data['target'].values

In [35]:
print(X) # 1st 3 and last 3

['awww bummer shoulda got david carr third day'
 'upset updat facebook text might cri result school today also blah'
 'dive mani time ball manag save rest go bound' ...
 'readi mojo makeov ask detail'
 'happi th birthday boo alll time tupac amaru shakur'
 'happi charitytuesday']


In [36]:
print(Y)

[0 0 0 ... 1 1 1]


In [38]:
#Split dataset into train and test set
X_train, X_test, Y_train, Y_test= train_test_split(X,Y, test_size=0.2, stratify= Y, random_state=2)

In [41]:
 print(X.shape, X_train.shape, X_test.shape) # 80-20

(1600000,) (1280000,) (320000,)


In [42]:
 print(X_train)

['watch saw iv drink lil wine' ''
 'even though favourit drink think vodka coke wipe mind time think im gonna find new drink'
 ... 'eager monday afternoon'
 'hope everyon mother great day wait hear guy store tomorrow'
 'love wake folger bad voic deeper']


In [43]:
print(X_test)

['fine much time chat twitter hubbi back summer amp tend domin free time'
 'ah may show w ruth kim amp geoffrey sanhueza'
 'mayb bay area thang dammit' ...
 'nevertheless hooray member wonder safe trip' 'feel well' 'thank']


In [44]:
#Convert textual data to numerical data
vectorizer=TfidfVectorizer() # assigns some importance to each word #checks if each word corresponds to +ve or -ve tweets

X_train=vectorizer.fit_transform(X_train) # understands vocab assigns score to each word
X_test=vectorizer.transform(X_test) # ensures test data is transformed using the same vocabulary and IDF values as training data. Ensures consistency and prevents overfitting.

In [45]:
print(X_train)

  (0, 183946)	0.4472729166586356
  (0, 95118)	0.42077260952923395
  (0, 45487)	0.37402636158338565
  (0, 82049)	0.5304215575897759
  (0, 142689)	0.3575144649296035
  (0, 180422)	0.27173150083210024
  (2, 113852)	0.167844893906494
  (2, 55900)	0.20286008573487765
  (2, 65029)	0.1879690549016539
  (2, 78757)	0.16312678769318012
  (2, 166359)	0.1516641535655483
  (2, 105934)	0.24141469878400337
  (2, 184143)	0.33473628844057507
  (2, 32612)	0.3129113979573425
  (2, 178596)	0.3295378319472821
  (2, 165149)	0.32096829349518735
  (2, 54355)	0.29063454343670403
  (2, 165485)	0.18702803756520983
  (2, 51531)	0.18915566286204807
  (2, 45487)	0.4589602892674482
  (3, 76624)	0.3745533706588445
  (3, 167264)	0.27084176678575755
  (3, 158679)	0.39418689933310996
  (3, 24202)	0.520082553063576
  (3, 65733)	0.2785086869326036
  :	:
  (1279996, 159427)	0.26121641201003176
  (1279996, 178842)	0.32194738864770095
  (1279996, 127502)	0.2516559547390209
  (1279996, 95807)	0.26529221192193436
  (1279996, 1

In [46]:
print(X_test)

  (0, 172105)	0.20140171998884335
  (0, 166359)	0.35223579612250294
  (0, 163193)	0.3911646206511155
  (0, 158492)	0.24646568739510374
  (0, 109784)	0.1997462992881365
  (0, 75935)	0.314408561649275
  (0, 58907)	0.26563073958272126
  (0, 55923)	0.286144248383845
  (0, 44133)	0.4120732680766349
  (0, 28663)	0.30048476892505444
  (0, 12218)	0.18207135448977682
  (0, 5989)	0.19281388551158826
  (1, 147882)	0.24276060641016736
  (1, 140657)	0.48201368218609736
  (1, 102534)	0.2840307561506464
  (1, 89108)	0.40706901280062907
  (1, 62637)	0.5715593366717205
  (1, 5989)	0.20841518091984773
  (1, 3086)	0.303701306490952
  (2, 164027)	0.5358238023125791
  (2, 102546)	0.31687648505369054
  (2, 38382)	0.44911880435001894
  (2, 14153)	0.4692099108344521
  (2, 8334)	0.43659650164567654
  (3, 97397)	0.5776951243753978
  :	:
  (319994, 89990)	0.367466721160491
  (319994, 84946)	0.3326141402317714
  (319994, 78788)	0.34541835256684617
  (319994, 51008)	0.45378985916820247
  (319995, 184949)	0.3199772

In [47]:
# Train ML model- Logistic Regression

model=LogisticRegression(max_iter=1000)

In [48]:
model.fit(X_train,Y_train)

LogisticRegression(max_iter=1000)

In [53]:
#Model Evaluation on training data

X_train_pred= model.predict(X_train)
training_data_accuracy= accuracy_score(Y_train, X_train_pred)

In [54]:
print('Accuracy of training data:' ,training_data_accuracy)

Accuracy of training data: 0.7924828125


In [55]:
#Model Evaluation on test data

X_test_pred= model.predict(X_test)
test_data_accuracy= accuracy_score(Y_test, X_test_pred)

In [56]:
print('Accuracy of testing data:' ,test_data_accuracy)


Accuracy of testing data: 0.774475
